# Unity Catalog Deep Dive: Advanced SQL Operations

This notebook provides a comprehensive guide to advanced Unity Catalog operations using SQL. We'll cover:

1. **Permissions Management** - Granting and managing access control
2. **Identity & Access Management** - Users, groups, and service principals
3. **Data Lineage** - Viewing and creating custom lineage
4. **Metric Views** - Creating and managing metric definitions

Each section includes both basic operations and advanced use cases for real-world scenarios.

## Prerequisites

- Unity Catalog enabled workspace
- Appropriate permissions to create and manage objects
- Access to system tables and information schema

## Configuration

Set up the catalog and schema we'll use throughout this notebook. You can modify these values for your environment.

In [ ]:
-- Configure the catalog and schema for this demo\n-- Modify these values for your environment\n\n-- Method 1: Direct replacement (recommended for notebooks)\n-- Just replace these values directly in the cells below\n-- CATALOG: users\n-- SCHEMA: ashwin_srikant\n\n-- Method 2: Session variables (if your cluster supports it)\n-- SET demo_catalog = 'users';\n-- SET demo_schema = 'ashwin_srikant';\n\n-- For this demo, we'll use direct values\nSELECT \n    'users' as catalog_name,\n    'ashwin_srikant' as schema_name,\n    'users.ashwin_srikant' as full_schema_name;"

-- Create a sample table for permission demonstrations\nCREATE OR REPLACE TABLE users.ashwin_srikant.product_orders (\n    order_id BIGINT,\n    customer_id BIGINT,\n    product_id STRING,\n    product_name STRING,\n    product_category STRING,\n    order_date DATE,\n    quantity_ordered INT,\n    unit_price DECIMAL(10,2),\n    total_order_value DECIMAL(12,2),\n    order_status STRING,\n    fulfillment_center STRING,\n    shipping_method STRING,\n    order_priority STRING\n);\n\n-- Insert sample data\nINSERT INTO users.ashwin_srikant.product_orders VALUES\n    (10001, 2001, 'PROD-001', 'Wireless Headphones', 'Electronics', '2024-01-15', 2, 129.99, 259.98, 'Shipped', 'FC-West', 'Standard', 'Normal'),\n    (10002, 2002, 'PROD-002', 'Standing Desk', 'Furniture', '2024-01-16', 1, 399.99, 399.99, 'Processing', 'FC-East', 'Express', 'High'),\n    (10003, 2003, 'PROD-003', 'Coffee Maker', 'Appliances', '2024-01-17', 1, 89.99, 89.99, 'Delivered', 'FC-Central', 'Standard', 'Normal'),\n    (10004, 2001, 'PROD-004', 'Laptop Bag', 'Accessories', '2024-01-18', 3, 45.50, 136.50, 'Cancelled', 'FC-West', 'Express', 'Low'),\n    (10005, 2004, 'PROD-001', 'Wireless Headphones', 'Electronics', '2024-01-19', 1, 129.99, 129.99, 'Shipped', 'FC-East', 'Overnight', 'High'),\n    (10006, 2005, 'PROD-005', 'Smart Watch', 'Electronics', '2024-01-20', 1, 249.99, 249.99, 'Processing', 'FC-Central', 'Standard', 'Normal'),\n    (10007, 2002, 'PROD-003', 'Coffee Maker', 'Appliances', '2024-01-21', 2, 89.99, 179.98, 'Delivered', 'FC-West', 'Express', 'Normal');\n\nSELECT 'Product orders sample table created with ' || COUNT(*) || ' order records' as status\nFROM users.ashwin_srikant.product_orders;"

In [ ]:
-- Create a sample table for permission demonstrations
CREATE OR REPLACE TABLE users.ashwin_srikant.product_orders (
    order_id BIGINT,
    customer_id BIGINT,
    product_id STRING,
    product_name STRING,
    product_category STRING,
    order_date DATE,
    quantity_ordered INT,
    unit_price DECIMAL(10,2),
    total_order_value DECIMAL(12,2),
    order_status STRING,
    fulfillment_center STRING,
    shipping_method STRING,
    order_priority STRING
);

-- Insert sample data
INSERT INTO users.ashwin_srikant.product_orders VALUES
    (10001, 2001, 'PROD-001', 'Wireless Headphones', 'Electronics', '2024-01-15', 2, 129.99, 259.98, 'Shipped', 'FC-West', 'Standard', 'Normal'),
    (10002, 2002, 'PROD-002', 'Standing Desk', 'Furniture', '2024-01-16', 1, 399.99, 399.99, 'Processing', 'FC-East', 'Express', 'High'),
    (10003, 2003, 'PROD-003', 'Coffee Maker', 'Appliances', '2024-01-17', 1, 89.99, 89.99, 'Delivered', 'FC-Central', 'Standard', 'Normal'),
    (10004, 2001, 'PROD-004', 'Laptop Bag', 'Accessories', '2024-01-18', 3, 45.50, 136.50, 'Cancelled', 'FC-West', 'Express', 'Low'),
    (10005, 2004, 'PROD-001', 'Wireless Headphones', 'Electronics', '2024-01-19', 1, 129.99, 129.99, 'Shipped', 'FC-East', 'Overnight', 'High'),
    (10006, 2005, 'PROD-005', 'Smart Watch', 'Electronics', '2024-01-20', 1, 249.99, 249.99, 'Processing', 'FC-Central', 'Standard', 'Normal'),
    (10007, 2002, 'PROD-003', 'Coffee Maker', 'Appliances', '2024-01-21', 2, 89.99, 179.98, 'Delivered', 'FC-West', 'Express', 'Normal');

SELECT 'Product orders sample table created with ' || COUNT(*) || ' order records' as status
FROM users.ashwin_srikant.product_orders;

## Granting Permissions on Different Object Types

In [ ]:
-- 1. Grant SELECT permission on a specific table
-- Note: Replace 'example_user@company.com' with actual user
-- GRANT SELECT ON TABLE users.ashwin_srikant.product_orders TO 'example_user@company.com';

-- 2. Grant USAGE permission on schema (required to access objects within)
-- GRANT USAGE ON SCHEMA users.ashwin_srikant TO 'example_user@company.com';

-- 3. Grant USAGE permission on catalog (required to access schemas within)
-- GRANT USAGE ON CATALOG users TO 'example_user@company.com';

-- 4. Grant permissions to a group instead of individual user
-- GRANT SELECT ON TABLE users.ashwin_srikant.product_orders TO 'order_analysts';

-- 5. Grant multiple permissions at once
-- GRANT SELECT, MODIFY ON TABLE users.ashwin_srikant.product_orders TO 'order_managers';

-- Display available permission types
SELECT 'Permission examples commented out - uncomment and modify for your environment' as note;

## Advanced Use Case: Role-Based Permission Management

Let's create a comprehensive permission structure for different roles in a data organization.

In [ ]:
-- Create additional objects for role-based permissions demo
CREATE OR REPLACE VIEW users.ashwin_srikant.order_performance_summary AS
SELECT 
    product_category,
    fulfillment_center,
    COUNT(*) as total_orders,
    SUM(total_order_value) as total_revenue,
    AVG(total_order_value) as avg_order_value,
    SUM(quantity_ordered) as total_units_sold
FROM users.ashwin_srikant.product_orders
WHERE order_status != 'Cancelled'
GROUP BY product_category, fulfillment_center;

-- Create a sensitive view (customer PII data)
CREATE OR REPLACE VIEW users.ashwin_srikant.customer_order_details AS
SELECT 
    customer_id,
    'customer_' || customer_id as customer_code,
    'email_' || customer_id || '@company.com' as customer_email,
    COUNT(*) as total_orders,
    SUM(total_order_value) as lifetime_value,
    AVG(total_order_value) as avg_order_value,
    MAX(order_date) as last_order_date,
    CASE 
        WHEN SUM(total_order_value) > 500 THEN 'VIP'
        WHEN SUM(total_order_value) > 200 THEN 'Premium' 
        ELSE 'Standard'
    END as customer_tier
FROM users.ashwin_srikant.product_orders
GROUP BY customer_id;

SELECT 'Created views for role-based permissions demo' as status;

In [ ]:
-- Role-Based Permission Strategy for Order Management
-- Uncomment and modify these grants for your environment

-- Order Analysts: Read-only access to performance summary data
-- GRANT USAGE ON CATALOG users TO 'order_analysts';
-- GRANT USAGE ON SCHEMA users.ashwin_srikant TO 'order_analysts';
-- GRANT SELECT ON VIEW users.ashwin_srikant.order_performance_summary TO 'order_analysts';

-- Order Managers: Full access to order data and ability to create objects
-- GRANT USAGE ON CATALOG users TO 'order_managers';
-- GRANT USAGE, CREATE ON SCHEMA users.ashwin_srikant TO 'order_managers';
-- GRANT SELECT, MODIFY ON TABLE users.ashwin_srikant.product_orders TO 'order_managers';
-- GRANT SELECT ON VIEW users.ashwin_srikant.order_performance_summary TO 'order_managers';

-- Data Scientists: Read access to order data for analysis and modeling
-- GRANT USAGE ON CATALOG users TO 'data_scientists';
-- GRANT USAGE ON SCHEMA users.ashwin_srikant TO 'data_scientists';
-- GRANT SELECT ON TABLE users.ashwin_srikant.product_orders TO 'data_scientists';
-- GRANT SELECT ON VIEW users.ashwin_srikant.order_performance_summary TO 'data_scientists';

-- Customer Service: Access to customer order details for support
-- GRANT USAGE ON CATALOG users TO 'customer_service';
-- GRANT USAGE ON SCHEMA users.ashwin_srikant TO 'customer_service';
-- GRANT SELECT ON VIEW users.ashwin_srikant.customer_order_details TO 'customer_service';

-- Fulfillment Teams: Access to order data by fulfillment center
-- GRANT USAGE ON CATALOG users TO 'fulfillment_teams';
-- GRANT USAGE ON SCHEMA users.ashwin_srikant TO 'fulfillment_teams';
-- GRANT SELECT ON TABLE users.ashwin_srikant.product_orders TO 'fulfillment_teams';

SELECT 'Role-based permission grants defined (commented out)' as note;

# 2. Identity & Access Management

Understanding users, groups, and service principals is crucial for effective access management in Unity Catalog.

## Viewing Current User Information

In [ ]:
-- Get current user information
SELECT 
    current_user() as current_user,
    current_catalog() as current_catalog,
    current_schema() as current_schema;

## Analyzing User Permissions on Specific Objects

In [ ]:
-- View permissions on our orders table
SHOW GRANTS ON TABLE users.ashwin_srikant.product_orders;

In [ ]:
-- View permissions on the schema
SHOW GRANTS ON SCHEMA users.ashwin_srikant;

In [ ]:
-- View permissions on the catalog
SHOW GRANTS ON CATALOG users;

## Advanced Use Case: Fine-Grained Access Control

Let's implement row and column-level security to show how different user groups see different views of the same data based on their permissions and group memberships.

In [ ]:
-- Step 1: Create column masking policies for sensitive data
-- Column-level security: Mask sensitive information based on group membership

-- Create a secure view that masks customer PII based on user groups
CREATE OR REPLACE VIEW users.ashwin_srikant.secure_customer_orders AS
SELECT 
    order_id,
    customer_id,
    
    -- Column masking: Show full customer details only to admins and executives
    CASE 
        WHEN is_member('admins') OR is_member('executives') THEN customer_id
        ELSE NULL
    END as visible_customer_id,
    
    product_id,
    product_name,
    product_category,
    order_date,
    quantity_ordered,
    unit_price,
    
    -- Revenue masking: Executives see revenue, analysts see ranges, others see NULL
    CASE 
        WHEN is_member('admins') OR is_member('executives') THEN total_order_value
        WHEN is_member('data_analysts') THEN 
            CASE 
                WHEN total_order_value < 100 THEN 'Under $100'
                WHEN total_order_value < 300 THEN '$100-$300'
                ELSE 'Over $300'
            END
        ELSE '***REDACTED***'
    END as masked_order_value,
    
    order_status,
    fulfillment_center,
    shipping_method,
    
    -- Priority masking: Only admins and operations see true priority
    CASE 
        WHEN is_member('admins') OR is_member('operations') THEN order_priority
        ELSE 'Standard'
    END as visible_priority
    
FROM users.ashwin_srikant.product_orders

-- Row-level security: Time-based filtering
WHERE 
    CASE 
        -- Admins see all historical data
        WHEN is_member('admins') THEN TRUE
        -- Other users only see last 30 days
        ELSE order_date >= CURRENT_DATE() - INTERVAL 30 DAYS
    END;

-- Test the secure view (results will vary based on your group membership)
SELECT 'Secure view created - data visibility depends on user group membership' as status;

In [ ]:
-- Step 2: Create role-specific views demonstrating data contracts

-- Executive View: Aggregated data with full financial visibility but no PII
CREATE OR REPLACE VIEW users.ashwin_srikant.executive_order_dashboard AS
SELECT 
    DATE_TRUNC('week', order_date) as week_ending,
    product_category,
    fulfillment_center,
    
    -- Full financial metrics for executives
    COUNT(*) as total_orders,
    SUM(total_order_value) as total_revenue,
    AVG(total_order_value) as avg_order_value,
    COUNT(DISTINCT customer_id) as unique_customers,
    
    -- Operational metrics
    COUNT(*) FILTER (WHERE order_status = 'Delivered') as delivered_orders,
    COUNT(*) FILTER (WHERE order_status = 'Cancelled') as cancelled_orders,
    ROUND(100.0 * COUNT(*) FILTER (WHERE order_status = 'Delivered') / COUNT(*), 2) as delivery_rate_pct,
    
    -- No individual customer data exposed
    'AGGREGATED' as data_privacy_level
    
FROM users.ashwin_srikant.secure_customer_orders
WHERE is_member('executives') OR is_member('admins')  -- Only executives and admins see this view
GROUP BY DATE_TRUNC('week', order_date), product_category, fulfillment_center;

-- Data Analyst View: Detailed operational data with masked financials
CREATE OR REPLACE VIEW users.ashwin_srikant.analyst_order_details AS
SELECT 
    order_id,
    order_date,
    product_name,
    product_category,
    quantity_ordered,
    
    -- Masked financial data - ranges instead of exact values
    masked_order_value as order_value_range,
    
    -- Full operational visibility
    order_status,
    fulfillment_center,
    shipping_method,
    visible_priority as priority,
    
    -- Time-based access control applied
    CASE 
        WHEN order_date >= CURRENT_DATE() - INTERVAL 7 DAYS THEN 'Recent'
        WHEN order_date >= CURRENT_DATE() - INTERVAL 30 DAYS THEN 'This Month'
        ELSE 'Historical' 
    END as recency_bucket,
    
    -- Customer data anonymized for analysts
    CASE 
        WHEN visible_customer_id IS NOT NULL THEN 'CUSTOMER_' || visible_customer_id
        ELSE 'ANONYMOUS'
    END as anonymized_customer
    
FROM users.ashwin_srikant.secure_customer_orders;

-- Show the different data contracts
SELECT 'Role-specific views created with different data access patterns' as status;

In [ ]:
-- Step 3: Demonstrate what different user groups see

-- Test the secure views to show access patterns
-- Note: Results depend on your actual group membership

-- What everyone sees from the base secure view (varies by group)
SELECT 
    'Secure Customer Orders - Base View' as view_name,
    COUNT(*) as visible_rows,
    COUNT(DISTINCT visible_customer_id) as visible_customers,
    MAX(order_date) as latest_order_date,
    MIN(order_date) as earliest_order_date
FROM users.ashwin_srikant.secure_customer_orders;

-- What executives see (aggregated, financial data, no PII)
SELECT 
    'Executive Dashboard' as view_name,
    SUM(total_orders) as total_orders_visible,
    SUM(total_revenue) as total_revenue_visible,
    COUNT(DISTINCT fulfillment_center) as fulfillment_centers,
    data_privacy_level
FROM users.ashwin_srikant.executive_order_dashboard
GROUP BY data_privacy_level;

-- What data analysts see (detailed operations, masked finance, time-limited)
SELECT 
    'Data Analyst View' as view_name,
    COUNT(*) as visible_order_records,
    COUNT(DISTINCT order_value_range) as revenue_buckets_visible,
    COUNT(DISTINCT recency_bucket) as time_periods_visible,
    MAX(order_date) as latest_data_date,
    MIN(order_date) as earliest_data_date
FROM users.ashwin_srikant.analyst_order_details;

## Fine-Grained Access Control Summary

The examples above demonstrate several key access control patterns:

### 🎭 **Column-Level Masking**
- **Financial Data**: Executives see exact values, analysts see ranges, others see redacted
- **Customer PII**: Only admins and executives see customer IDs, others get NULL
- **Operational Details**: Priority levels visible only to operations and admin teams

### ⏰ **Row-Level Security (Time-Based)**
- **Admins**: See all historical data (no time restrictions)
- **Other Users**: Limited to last 30 days of order data
- **Dynamic Filtering**: Applied at query time based on group membership

### 👥 **Role-Based Data Contracts**

| **User Group** | **Financial Data** | **Customer Data** | **Time Access** | **Detail Level** |
|---|---|---|---|---|
| **Admins** | Full amounts | Full customer IDs | All history | Complete access |
| **Executives** | Full amounts | Aggregated only | Last 30 days | Strategic metrics |
| **Data Analysts** | Bucketed ranges | Anonymized | Last 30 days | Operational details |
| **Other Users** | Redacted | Redacted | Last 30 days | Basic info only |

### 🔒 **Implementation Benefits**
- **Single Source of Truth**: One table, multiple secure views
- **Dynamic Security**: Access changes based on group membership
- **Audit Trail**: All access patterns logged and traceable
- **Performance**: Security applied at view level, not query level
- **Compliance**: Built-in data governance for regulatory requirements

This approach ensures that users only see data appropriate for their role while maintaining operational efficiency.

## Group Membership Analysis

In [ ]:
-- Query to see group memberships (requires appropriate permissions)
-- Note: This may require admin privileges to see all groups

-- List all groups in the workspace
SELECT 
    id as group_id,
    display_name as group_name,
    external_id
FROM system.access.groups
ORDER BY display_name
LIMIT 10;

In [ ]:
-- List users and their group memberships
SELECT 
    u.id as user_id,
    u.user_name,
    u.display_name as user_display_name,
    g.display_name as group_name
FROM system.access.users u
LEFT JOIN system.access.group_members gm ON u.id = gm.member_id
LEFT JOIN system.access.groups g ON gm.group_id = g.id
WHERE u.active = true
ORDER BY u.user_name, g.display_name
LIMIT 20;

# 3. Data Lineage Management

Unity Catalog automatically tracks lineage for many operations, but you can also add custom lineage information for external processes.

## Viewing Automatic Lineage

In [ ]:
-- View table lineage information
SELECT 
    source_table_full_name,
    source_column_name,
    target_table_full_name,
    target_column_name,
    created_at,
    created_by
FROM system.access.table_lineage 
WHERE target_table_full_name LIKE 'users.ashwin_srikant.%'
ORDER BY created_at DESC
LIMIT 10;

In [ ]:
-- View column-level lineage
SELECT 
    source_table_full_name,
    source_column_name,
    target_table_full_name, 
    target_column_name,
    created_at
FROM system.access.column_lineage
WHERE target_table_full_name LIKE 'users.ashwin_srikant.%'
   OR source_table_full_name LIKE 'users.ashwin_srikant.%'
ORDER BY created_at DESC
LIMIT 10;

## Bring Your Own Lineage (BYOL)

For external processes or ETL tools, you can add custom lineage information to Unity Catalog.

In [ ]:
-- Create source and target tables for order processing lineage demonstration
CREATE OR REPLACE TABLE users.ashwin_srikant.raw_order_events (
    event_id BIGINT,
    order_id BIGINT,
    customer_id BIGINT,
    product_id STRING,
    event_type STRING,
    event_timestamp TIMESTAMP,
    quantity INT,
    unit_price DECIMAL(10,2),
    source_system STRING
);

CREATE OR REPLACE TABLE users.ashwin_srikant.processed_order_metrics (
    order_id BIGINT,
    customer_id BIGINT,
    total_order_value DECIMAL(12,2),
    order_processing_time_minutes INT,
    fulfillment_status STRING,
    customer_segment STRING,
    processed_at TIMESTAMP
);

-- Insert sample data
INSERT INTO users.ashwin_srikant.raw_order_events VALUES
    (1, 10001, 2001, 'PROD-001', 'order_placed', '2024-01-15 10:00:00', 2, 129.99, 'Web'),
    (2, 10001, 2001, 'PROD-001', 'payment_confirmed', '2024-01-15 10:05:00', 2, 129.99, 'Payment Gateway'),
    (3, 10001, 2001, 'PROD-001', 'order_shipped', '2024-01-15 14:30:00', 2, 129.99, 'Warehouse System');

SELECT 'Created tables for order processing lineage demonstration' as status;

## Adding Custom Lineage via Python API

Unity Catalog provides a REST API for adding custom lineage information. This is the recommended approach for external ETL tools, data pipelines, and custom applications.

In [ ]:
# Example: Adding Custom Lineage for External Systems via Python API

import requests
import json
from datetime import datetime

# Configuration for Unity Catalog Lineage API
def add_custom_lineage():
    """
    Add custom lineage information for external systems and downstream consumers.
    This demonstrates BYOL (Bring Your Own Lineage) for:
    - Upstream: xdb and Iris source systems
    - Downstream: Thoughtspot and Tableau consumers
    """
    
    # Get workspace URL and token (these would be set in your environment)
    workspace_url = dbutils.secrets.get("lineage", "workspace_url")  # e.g., "https://your-workspace.databricks.com"
    access_token = dbutils.secrets.get("lineage", "access_token")
    
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
    
    # Define our Unity Catalog tables
    demo_catalog = "users"
    demo_schema = "ashwin_srikant" 
    
    # 1. UPSTREAM LINEAGE: External source systems to Unity Catalog
    upstream_lineage_entries = [
        {
            # xdb order_events table -> raw_order_events
            "source": {
                "table_full_name": "xdb.production.order_events",
                "table_type": "EXTERNAL"
            },
            "targets": [
                {
                    "table_full_name": f"{demo_catalog}.{demo_schema}.raw_order_events",
                    "table_type": "TABLE"
                }
            ]
        },
        {
            # Iris CRM system -> customer_order_details 
            "source": {
                "table_full_name": "iris.crm.customers",
                "table_type": "EXTERNAL"
            },
            "targets": [
                {
                    "table_full_name": f"{demo_catalog}.{demo_schema}.customer_order_details",
                    "table_type": "VIEW"
                }
            ]
        }
    ]
    
    # 2. DOWNSTREAM LINEAGE: Unity Catalog to BI Tools
    downstream_lineage_entries = [
        {
            # order_performance_summary -> Thoughtspot
            "source": {
                "table_full_name": f"{demo_catalog}.{demo_schema}.order_performance_summary",
                "table_type": "VIEW"
            },
            "targets": [
                {
                    "table_full_name": "thoughtspot.dashboards.order_operations_dashboard",
                    "table_type": "EXTERNAL"
                }
            ]
        },
        {
            # advanced_order_metrics -> Tableau
            "source": {
                "table_full_name": f"{demo_catalog}.{demo_schema}.advanced_order_metrics",
                "table_type": "VIEW"
            },
            "targets": [
                {
                    "table_full_name": "tableau.workbooks.executive_order_reporting",
                    "table_type": "EXTERNAL"
                }
            ]
        }
    ]
    
    print("=== LINEAGE STRUCTURE ===")
    print("📥 UPSTREAM SOURCES:")
    print("  • xdb.production.order_events → raw_order_events")
    print("  • iris.crm.customers → customer_order_details")
    print("\n📊 DOWNSTREAM CONSUMERS:")
    print("  • order_performance_summary → Thoughtspot Dashboards")
    print("  • advanced_order_metrics → Tableau Executive Reports")

# Execute the lineage demonstration
add_custom_lineage()

## Advanced Use Case: Complete Lineage Analysis

Let's create a comprehensive lineage analysis that shows the complete data flow for our schema.

In [ ]:
-- Comprehensive lineage analysis query
WITH recursive_lineage AS (
    -- Base case: direct lineage relationships
    SELECT 
        source_table_full_name,
        target_table_full_name,
        1 as lineage_depth,
        ARRAY(source_table_full_name, target_table_full_name) as lineage_path
    FROM system.access.table_lineage
    WHERE source_table_full_name LIKE 'users.ashwin_srikant.%'
       OR target_table_full_name LIKE 'users.ashwin_srikant.%'
),
lineage_summary AS (
    SELECT 
        source_table_full_name,
        target_table_full_name,
        lineage_depth,
        COUNT(*) as relationship_count
    FROM recursive_lineage
    GROUP BY source_table_full_name, target_table_full_name, lineage_depth
)
SELECT 
    CASE 
        WHEN source_table_full_name IS NULL THEN 'ROOT SOURCE'
        ELSE source_table_full_name 
    END as source_table,
    CASE 
        WHEN target_table_full_name IS NULL THEN 'FINAL TARGET'
        ELSE target_table_full_name 
    END as target_table,
    lineage_depth,
    relationship_count
FROM lineage_summary
ORDER BY lineage_depth, source_table_full_name, target_table_full_name;

In [ ]:
-- Impact analysis: What tables would be affected if we changed the orders table?
WITH downstream_impact AS (
    SELECT DISTINCT
        'users.ashwin_srikant.product_orders' as source_table,
        target_table_full_name as affected_table,
        'Direct dependency' as impact_type
    FROM system.access.table_lineage 
    WHERE source_table_full_name = 'users.ashwin_srikant.product_orders'
    
    UNION ALL
    
    SELECT DISTINCT
        'users.ashwin_srikant.product_orders' as source_table,
        l2.target_table_full_name as affected_table,
        'Indirect dependency (2 levels)' as impact_type
    FROM system.access.table_lineage l1
    JOIN system.access.table_lineage l2 ON l1.target_table_full_name = l2.source_table_full_name
    WHERE l1.source_table_full_name = 'users.ashwin_srikant.product_orders'
)
SELECT 
    source_table,
    affected_table,
    impact_type,
    COUNT(*) as dependency_count
FROM downstream_impact
GROUP BY source_table, affected_table, impact_type
ORDER BY impact_type, affected_table;

# 4. Metric Views

Metric views in Unity Catalog provide a way to define reusable business metrics with consistent definitions across your organization.

## Basic Metric View Creation

In [ ]:
-- Create a basic metric view for order metrics
CREATE OR REPLACE VIEW users.ashwin_srikant.order_metrics AS
SELECT 
    -- Time dimensions
    order_date,
    YEAR(order_date) as order_year,
    MONTH(order_date) as order_month,
    QUARTER(order_date) as order_quarter,
    
    -- Operational dimensions
    fulfillment_center,
    shipping_method,
    order_priority,
    
    -- Product dimensions
    product_category,
    product_name,
    
    -- Core metrics
    COUNT(*) as total_orders,
    SUM(total_order_value) as total_revenue,
    AVG(total_order_value) as average_order_value,
    MIN(total_order_value) as min_order_value,
    MAX(total_order_value) as max_order_value,
    STDDEV(total_order_value) as revenue_std_dev,
    
    -- Quantity metrics
    SUM(quantity_ordered) as total_units_ordered,
    AVG(quantity_ordered) as avg_units_per_order,
    
    -- Derived metrics
    SUM(total_order_value) / COUNT(DISTINCT customer_id) as revenue_per_customer,
    COUNT(DISTINCT customer_id) as unique_customers,
    COUNT(*) / COUNT(DISTINCT customer_id) as orders_per_customer
    
FROM users.ashwin_srikant.product_orders
WHERE order_status != 'Cancelled'
GROUP BY 
    order_date, 
    YEAR(order_date),
    MONTH(order_date), 
    QUARTER(order_date),
    fulfillment_center,
    shipping_method,
    order_priority,
    product_category, 
    product_name;

-- Test the metric view
SELECT * FROM users.ashwin_srikant.order_metrics
ORDER BY order_date, fulfillment_center, product_category;

## Advanced Metric View with Time Intelligence

In [ ]:
-- Create an advanced order metric view with time intelligence and window functions
CREATE OR REPLACE VIEW users.ashwin_srikant.advanced_order_metrics AS
WITH daily_orders AS (
    SELECT 
        order_date,
        fulfillment_center,
        product_category,
        SUM(total_order_value) as daily_revenue,
        COUNT(*) as daily_order_count,
        COUNT(DISTINCT customer_id) as daily_unique_customers,
        AVG(total_order_value) as daily_avg_order_value
    FROM users.ashwin_srikant.product_orders
    WHERE order_status != 'Cancelled'
    GROUP BY order_date, fulfillment_center, product_category
),
time_series_metrics AS (
    SELECT 
        order_date,
        fulfillment_center,
        product_category,
        daily_revenue,
        daily_order_count,
        daily_unique_customers,
        daily_avg_order_value,
        
        -- Moving averages (3-day window)
        AVG(daily_revenue) OVER (
            PARTITION BY fulfillment_center, product_category 
            ORDER BY order_date 
            ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
        ) as revenue_3day_ma,
        
        AVG(daily_order_count) OVER (
            PARTITION BY fulfillment_center, product_category 
            ORDER BY order_date 
            ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
        ) as orders_3day_ma,
        
        -- Period-over-period changes
        LAG(daily_revenue, 1) OVER (
            PARTITION BY fulfillment_center, product_category 
            ORDER BY order_date
        ) as prev_day_revenue,
        
        LAG(daily_order_count, 1) OVER (
            PARTITION BY fulfillment_center, product_category 
            ORDER BY order_date
        ) as prev_day_orders,
        
        -- Cumulative metrics
        SUM(daily_revenue) OVER (
            PARTITION BY fulfillment_center, product_category 
            ORDER BY order_date 
            ROWS UNBOUNDED PRECEDING
        ) as cumulative_revenue,
        
        -- Ranking within fulfillment center
        RANK() OVER (
            PARTITION BY fulfillment_center, order_date 
            ORDER BY daily_revenue DESC
        ) as category_revenue_rank
        
    FROM daily_orders
)
SELECT 
    *,
    -- Calculated performance metrics
    CASE 
        WHEN prev_day_revenue > 0 THEN 
            ((daily_revenue - prev_day_revenue) / prev_day_revenue) * 100
        ELSE NULL 
    END as revenue_change_pct,
    
    CASE 
        WHEN prev_day_orders > 0 THEN 
            ((daily_order_count - prev_day_orders) / prev_day_orders) * 100
        ELSE NULL 
    END as order_count_change_pct,
    
    -- Performance indicators
    CASE 
        WHEN daily_revenue > revenue_3day_ma THEN 'Above Average'
        WHEN daily_revenue < revenue_3day_ma THEN 'Below Average'
        ELSE 'Average'
    END as revenue_performance_indicator,
    
    CASE 
        WHEN daily_order_count > orders_3day_ma THEN 'Above Average'
        WHEN daily_order_count < orders_3day_ma THEN 'Below Average'
        ELSE 'Average'
    END as volume_performance_indicator
    
FROM time_series_metrics;

-- Test the advanced metric view
SELECT * FROM users.ashwin_srikant.advanced_order_metrics
ORDER BY fulfillment_center, product_category, order_date;

## Metric View Documentation and Metadata

In [ ]:
-- Add comments and metadata to our order metric views
COMMENT ON VIEW users.ashwin_srikant.order_metrics IS 
'Basic order metrics aggregated by date, fulfillment center, and product category. 
Includes revenue, order counts, and customer metrics for operational analysis.';

COMMENT ON VIEW users.ashwin_srikant.advanced_order_metrics IS 
'Advanced order metrics with time intelligence features including moving averages, 
period-over-period comparisons, and performance indicators for trend analysis.';

-- Query to see all metric views and their documentation
SELECT 
    table_name as metric_view_name,
    comment as description,
    created as created_date
FROM system.information_schema.tables 
WHERE table_catalog = 'users' 
    AND table_schema = 'ashwin_srikant'
    AND table_type = 'VIEW'
    AND (table_name LIKE '%metrics%' OR table_name LIKE '%metric%')
ORDER BY created DESC;

# Summary and Best Practices

This notebook demonstrated advanced Unity Catalog operations across four key areas:

## 🔐 Permissions Management
- **Hierarchical permissions**: Catalog → Schema → Object level
- **Role-based access control**: Define permissions by user roles
- **Principle of least privilege**: Grant only necessary permissions

## 👥 Identity & Access Management
- **Comprehensive auditing**: Track who has access to what
- **Group management**: Use groups for scalable permission management
- **Regular access reviews**: Monitor and audit permissions regularly

## 🔄 Data Lineage
- **Automatic tracking**: Unity Catalog tracks most lineage automatically
- **Custom lineage**: Add BYOL for external processes
- **Impact analysis**: Understand downstream effects of changes

## 📊 Metric Views
- **Consistent definitions**: Centralize business metric definitions
- **Multi-dimensional analysis**: Support various business dimensions
- **Documentation**: Always document metric calculations and business logic

## 🚀 Next Steps

1. **Implement governance policies** using the permission patterns shown
2. **Set up regular access audits** using the identity management queries
3. **Create lineage documentation** for all critical data flows
4. **Build a metric catalog** using the framework patterns demonstrated
5. **Automate monitoring** of permissions, lineage, and metric quality

Remember to adapt these patterns to your specific organizational needs and compliance requirements!